<font face="Courier" size=7 >Maxwell Activity Scanner - Source Code

In this notebook we develop an **activity scanner** for the maxwell system. The purpose of an activity scanner is to determine witch electrodes are in contact with healthy electricaly active neurons. This code replicates the functionality of the activity scanner that can be found inside Maxwell's GUI application, **Scope**.

**References**
* Read the [Maxwell Manual](MaxLabLive_Manual.pdf) to learn about the original activity scanner on Maxwell.
* Check out the original Maxwell activity scanner by opening the Maxwell app, Scope, inside of Teamviewer

<font color="orange">

**Check**
* Does `recored_spikes_only` work?
* Sanity check that recording function works

<font color="red">

**To Do**
* Do analysis of resulting recordings to find healthy neurons
* Get actuall total for scan time


# <font color="grey">Set Up Notebook 

Import required maxlab packages

In [1]:
import ipywidgets as ipw

In [2]:
import maxlab
import maxlab.system
import maxlab.chip
import maxlab.util
import maxlab.saving

Import generic python packages

In [3]:
import os
import tempfile
import time

Set port for communication

In [4]:
#port = 7215

## <font color="gray">Helper function

This function is sometimes userful when debugging code in the <font color="magenta">Electrode Configurater Section</font>. It is not used in the final app at all

In [5]:
def numToGrid(num):
    return num//220, num%220

In [6]:
#numToGrid(226)

# <font color="blue">recordActivity Function 

<font color="blue">recordActivity</font> does one recording from a selection of electrodes that is passed into the function.

The code in this section originally came from [this notebook](/notebooks/Projects/Maxwell_Controller/recorder/Maxwell%20Recorder%20Source%20Code.ipynb)

In [7]:
def recordActivity(electrodes, recording_file_name="scan", recording_length=20, gain=512, record_only_spikes=True,
                   data_path="/home/mxwbio/wetai/homepage/Projects/Maxwell_Activity_Scanner/data"):
    
    #print("recording electrodes for "+str(recording_length)+" seconds")
    maxlab.util.initialize()                                # Initialize Maxwell
    maxlab.send( maxlab.chip.Amplifier().set_gain(gain) )   # Set Gain
    
    array = maxlab.chip.Array('online')     # Load Electrodes
    array.reset()
    array.select_electrodes( electrodes )
    array.route()                           #This might be necessary, but not sure
    array.download()
    maxlab.util.offset()   
    
    s = maxlab.saving.Saving()             # Set up file and wells for recording, 
    s.open_directory(data_path)            # I don't fully understand this code, it's taken from an example
    s.set_legacy_format(True)
    s.group_delete_all()
    
    if not record_only_spikes:             # start recording and save results
        s.group_define(0, "routed")
    s.start_file(recording_file_name)
    #print("Recording Started")
    s.start_recording( range(1) )
    time.sleep(recording_length)
    #print("Saving Results")
    s.stop_recording()
    s.stop_file()
    s.group_delete_all()
    #print("Finished")

In [8]:
#recordActivity(list(range(1024)))

# <font color="magenta">Eletrode Configurator

## <font color="gray">Notes

There are 26400 electrodes on the MaxOne arranged in a $220\times120$ grid. Below we sho how the electrodes are numberd on the 4 corners of the HTML canvas grid.

Top Left:0

Bottom Left: 26180

Top Right: 219

Bottom Right: 26399

## <font color="gray">Get Electrode Scans

In [9]:
def getScansHelper(space=6):
    scans = []

    for right_shift in range(space): # How many indices we shift right before we start the scan
        for down_shift in range(space):

            electrodes = []
            most_left = down_shift * 220               #  index that is at the left edge of hte grid for a given row
            i_left = right_shift + down_shift * 220    # leftmost index in a row where we collect electrodes

            while i_left/220 < 120:              # check that the row which we want to collect electrodes from is not beyond bottom boundary
                i = i_left                         # Append leftmost electrode to list, then start moving right to collect more
                electrodes.append(i_left)

                while i-most_left+space < 220:  # keep moving right until we reach the rightmost boundary 
                    i += space
                    electrodes.append(i)

                i_left += 220*space         # once we reach the rightmost boundary, jump down to the next row
                most_left += 220*space

            scans.append(electrodes)        # add scan to list of scans
    return scans

## <font color="magenta">Optimized All Scans

In [10]:
def getScans():
    scans = getScansHelper(5)
    new_scan = []
    starter = 1055
    for scan in scans:
        for i in range(starter,0,-33):
            new_scan.append( scan.pop(i) )
        starter += -1
    scans.append(new_scan)
    return scans

## <font color="brown">Scratch Paper

<font color="brown">Validate data look correct

In [11]:
#scans = getAllScans()

In [12]:
# all_scans = []
# for scan in scans:
#     all_scans += scan

In [13]:
 #all_scans.sort()

# print(all_scans)

# len(scans)

# print(scans[1])

# len(all_scans)

#import numpy as np
# len(np.unique(all_scans))

# for scan in scans:
#     print(len(scan))

#numToGrid( 23979)

# for i in electrodes:
#     print(numToGrid(i))

# <font color="green">Run 1st Scanner

Here we combine the code `recordActivity` function with `getScans` to scan all electrodes.

## <font color="green">Build widgets

In [14]:
#wDataPath = ipw.Text(description="Data Path", value="/home/mxwbio/wetai/homepage/Projects/Maxwell_Activity_Scanner/data")
wGain1 = ipw.IntSlider( description='Gain',min=1,max=1024,value=512)
wRecordingLength1 = ipw.IntText( value=30, layout=ipw.Layout(width='auto'))
wRunScan1 = ipw.Button(description="Run", button_style="success" ) # , layout=ipw.Layout(width='auto')
wLengthBox1 = ipw.HBox([ipw.HTML("<p>Recording Lengths (seconds)</p>"),wRecordingLength1 ])
wMessage1 = ipw.HTML("<h4>Total Scan Time: 25 minutes</h4>")

In [15]:
ScannerBox1 = ipw.VBox([ wGain1, wLengthBox1, wMessage1 , wRunScan1])
ScannerBox1

## <font color="green">Add Function

Update message to user based on the value of the `wRecordingLength` widget

In [16]:
def updateMessage1(change):
    wMessage1.value = "<h4>Total Scan Time: "+str(round( wRecordingLength.value*26/60, 1 ))+" minutes</h4>"
wRecordingLength1.observe(updateMessage1, names='value')

Run scan was user clicks `wRunScan` button

In [18]:
def runScan1(b):
    
    # If previous activity scan is already there, delete the files
    files = !cd /home/mxwbio/wetai/homepage/Projects/Maxwell_Activity_Scanner/scan1_data && ls
    if len(files):
        !cd /home/mxwbio/wetai/homepage/Projects/Maxwell_Activity_Scanner/scan1_data && rm *
    
    # do activity scan
    count = 0                
    for scan in getScans():
        count +=1
        print("Recording "+str(count)+" of 26")
        recordActivity(scan, recording_length= wRecordingLength1.value, gain=wGain1.value)
    print("Finished!")
wRunScan1.on_click(runScan1)

# <font color="DarkGreen">Run 2nd Scanner

## <font color="DarkGreen"> Build Widgets

In [30]:
#wDataPath = ipw.Text(description="Data Path", value="/home/mxwbio/wetai/homepage/Projects/Maxwell_Activity_Scanner/data")
wGain2 = ipw.IntSlider( description='Gain',min=1,max=1024,value=512)
wRecordingLength2 = ipw.IntText( value=30, layout=ipw.Layout(width='auto'))
wRunScan2 = ipw.Button(description="Run", button_style="success" ) # , layout=ipw.Layout(width='auto')
wLengthBox2 = ipw.HBox([ipw.HTML("<p>Recording Lengths (seconds)</p>"),wRecordingLength2 ])
wMessage2 = ipw.HTML("<h4>Total Scan Time: 1 minutes</h4>")

In [31]:
ScannerBox2 = ipw.VBox([ wGain2, wLengthBox2, wMessage2, wRunScan2])
ScannerBox2

## <font color="DarkGreen"> Add Function

Update message to user based on the value of the `wRecordingLength` widget

In [32]:
def updateMessage2(change):
    wMessage2.value = "<h4>Total Scan Time: "+str(round( wRecordingLength2.value/60, 1 ))+" minutes</h4>"
wRecordingLength2.observe(updateMessage2, names='value')

Run scan was user clicks `wRunScan` button

In [33]:
def runScan2(b):
    
    # If previous activity scan is already there, delete the files
    files = !cd /home/mxwbio/wetai/homepage/Projects/Maxwell_Activity_Scanner/scan2_data && ls
    if len(files):
        !cd /home/mxwbio/wetai/homepage/Projects/Maxwell_Activity_Scanner/scan2_data && rm *
    
    recordActivity(electrodes_record , recording_length= wRecordingLength2.value, gain=wGain2.value)

wRunScan2.on_click(runScan2)

# <font color="peru">Analyze Results <font color="red">Not Done

# <font color="brown">Scratch Paper